In [56]:
using Base.Math
using DataFrames
using LinearAlgebra
using LinearAlgebra.BLAS
import CSV

In [57]:
input_file_path = "../data/shot_data.csv"
output_file_path = "../data/shot_data_aug.csv"

"../data/shot_data_aug.csv"

In [58]:
df = DataFrame(CSV.File(file_path))

Row,location_x,location_y,body_part,outcome,type
,Float64,Float64,String15,String31,String15
1,111.5,52.9,Right Foot,Off T,Open Play
2,113.9,26.4,Left Foot,Off T,Open Play
3,93.7,34.7,Left Foot,Saved,Open Play
4,109.2,39.1,Head,Off T,Open Play
5,107.8,24.7,Right Foot,Off T,Open Play
6,108.6,27.8,Left Foot,Wayward,Open Play
7,112.5,41.7,Head,Off T,Open Play
8,112.4,41.0,Head,Off T,Open Play
9,102.5,30.1,Right Foot,Saved,Open Play


In [59]:
goal_width = 8
goal_post_center = [120.0, 40.0]
goal_post_edges = [
    [goal_post_center[1], goal_post_center[2] - goal_width / 2], 
    [goal_post_center[1], goal_post_center[2] + goal_width / 2]
]

2-element Vector{Vector{Float64}}:
 [120.0, 36.0]
 [120.0, 44.0]

In [60]:
delta_x = df.location_x .- goal_post_center[1]
delta_y = df.location_y .- goal_post_center[2]
distance = .√((delta_x .* delta_x) .+ (delta_y .* delta_y))

31195-element Vector{Float64}:
 15.448624534242521
 14.905368160498417
 26.828715958837833
 10.837435120913064
 19.568597292601225
 16.69730517179344
  7.6902535718921525
  7.665507158694714
 20.106217943710845
 15.852129194527782
 15.74706321826391
  9.470480452437458
 21.600231480241135
  ⋮
 26.462237244798484
 33.63406011768428
 24.922680433693326
 10.868302535354815
 14.921461054467825
 11.62497311824849
 10.475208828467334
 18.752333188166215
 30.034147232774902
 17.626400653565096
 19.44865033877673
 23.574774654278247

In [61]:
function angle_subtended(p_vec :: Vector{Float64}, l_vec :: Vector{Float64}, r_vec :: Vector{Float64}) :: Float64
    # using a . b == ||a|| ||b|| cos(theta) <=> cos(theta) = (a . b) / (||a|| ||b||)
    return rad2deg(acos(dot(l_vec, r_vec) / (nrm2(l_vec) * nrm2(r_vec))))
end

angle_subtended (generic function with 2 methods)

In [62]:
angle_subtended_vec = []
for (x, y) in zip(df.location_x, df.location_y)
    p_vec = [x, y]
    l_vec = goal_post_edges[1] .- p_vec
    r_vec = goal_post_edges[2] .- p_vec
    push!(angle_subtended_vec, angle_subtended(p_vec, l_vec, r_vec))
end

angle_subtended_vec

31195-element Vector{Any}:
 16.98258565406719
 13.316705918518315
 16.6444064594191
 40.41941129511731
 14.895257080503281
 19.138408720284705
 54.28386495368449
 54.88168326501548
 19.82844769298744
 18.778338811577775
 25.61177021289632
 20.805296281122256
 14.341920848310371
  ⋮
 14.863643327346175
 13.31781023674157
 13.743137612784043
 39.70609167690597
 22.500654904068533
 32.53400635634941
 39.91063144744059
 24.022646156907438
 13.895417381608052
 24.12895958909258
 19.34814908062756
 10.98765796968888

In [63]:
insertcols!(df, :distance => distance, :degrees_subtended => angle_subtended_vec)

Row,location_x,location_y,body_part,outcome,type,distance,degrees_subtended
,Float64,Float64,String15,String31,String15,Float64,Any
1,111.5,52.9,Right Foot,Off T,Open Play,15.4486,16.9826
2,113.9,26.4,Left Foot,Off T,Open Play,14.9054,13.3167
3,93.7,34.7,Left Foot,Saved,Open Play,26.8287,16.6444
4,109.2,39.1,Head,Off T,Open Play,10.8374,40.4194
5,107.8,24.7,Right Foot,Off T,Open Play,19.5686,14.8953
6,108.6,27.8,Left Foot,Wayward,Open Play,16.6973,19.1384
7,112.5,41.7,Head,Off T,Open Play,7.69025,54.2839
8,112.4,41.0,Head,Off T,Open Play,7.66551,54.8817
9,102.5,30.1,Right Foot,Saved,Open Play,20.1062,19.8284


In [64]:
CSV.write(output_file_path, df)

"../data/shot_data_aug.csv"